# Spark 아키텍처 & DataFrame 실무
- Spark 실행 구조 (Driver/Executor), Lazy Evaluation
- DataFrame 중심 처리, CSV vs Parquet
- 산출물: 스키마 정의 코드

In [1]:
# Colab: TestData 폴더를 /content/에 업로드하세요.
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

# Colab 환경인지 확인
IN_COLAB = "google.colab" in sys.modules
# Colab이면 /content, 아니면 현재 작업 디렉토리를 BASE로 설정
BASE = "/content" if IN_COLAB else os.getcwd()

# CSV 파일 경로 설정
CSV_PATH = os.path.join(BASE, "titanic.csv")
# 출력할 Parquet 파일 경로 설정
OUTPUT_PARQUET = os.path.join(BASE, "out_titanic.parquet")

# SparkSession 생성 (DataFrame 및 Parquet 처리를 위한 애플리케이션)
spark = SparkSession.builder.appName("DataFrame_Parquet").getOrCreate()

### 스키마 명시 정의

In [2]:
# Titanic 데이터셋의 스키마 정의
titanic_schema = StructType([
    StructField("PassengerId", IntegerType(), True),  # 승객 ID (정수형, Null 허용)
    StructField("Survived", IntegerType(), True),     # 생존 여부 (0: 사망, 1: 생존)
    StructField("Pclass", IntegerType(), True),       # 객실 등급 (1, 2, 3등석)
    StructField("Name", StringType(), True),          # 승객 이름
    StructField("Sex", StringType(), True),           # 성별
    StructField("Age", DoubleType(), True),           # 나이 (실수형)
    StructField("SibSp", IntegerType(), True),        # 함께 탑승한 형제자매/배우자 수
    StructField("Parch", IntegerType(), True),        # 함께 탑승한 부모/자녀 수
    StructField("Ticket", StringType(), True),        # 티켓 번호
    StructField("Fare", DoubleType(), True),          # 요금 (실수형)
    StructField("Cabin", StringType(), True),         # 객실 번호
    StructField("Embarked", StringType(), True),      # 탑승 항구 (C, Q, S)
])

CSV 읽기 (스키마 명시)

In [3]:
# CSV 파일을 DataFrame으로 읽기
df_csv = (spark.read.format("csv")  # CSV 형식 지정
    .option("header", "true")        # 첫 번째 행을 헤더로 사용
    .option("sep", ",")              # 구분자를 쉼표로 설정
    .schema(titanic_schema)          # 미리 정의된 스키마 적용
    .load(CSV_PATH))                 # CSV 파일 로드

# 상위 5개 행 출력
df_csv.limit(5).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

DataFrame 기본 작업: 스키마, 데이터 타입, 통계

In [4]:
# 스키마 확인
df_csv.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [5]:
# 컬럼명과 데이터 타입 (Python 리스트)
df_csv.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [6]:
# 수치형 요약 통계 (count, mean, stddev, min, max)
df_csv.describe("Age", "Fare").show()

+-------+------------------+-----------------+
|summary|               Age|             Fare|
+-------+------------------+-----------------+
|  count|               714|              891|
|   mean| 29.69911764705882| 32.2042079685746|
| stddev|14.526497332334035|49.69342859718089|
|    min|              0.42|              0.0|
|    max|              80.0|         512.3292|
+-------+------------------+-----------------+



In [7]:
# 요약 통계 (count, mean, stddev, min, 25%, 50%, 75%, max)
df_csv.summary().toPandas()

,summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,count,891,891,891,891,891,714,891,891,891,891,204,889
1,mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,None,None
2,stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,None,None
3,min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0.0,A10,C
4,25%,223,0,2,None,None,20.0,0,0,19996.0,7.8958,None,None
5,50%,446,0,3,None,None,28.0,0,0,236171.0,14.4542,None,None
6,75%,669,1,3,None,None,38.0,1,0,347743.0,31.0,None,None
7,max,891,1,3,"van Melkebeke, Mr. Philemon",male,80.0,8,6,WE/P 5735,512.3292,T,S


Data Filtering: 조건에 맞는 행만 선택

In [8]:
# Survived == 1 인 행만
df_csv.filter(F.col("Survived") == 1).limit(5).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          9|       1|     3|Johnson, Mrs. Osc...|female|27.0|    0|    2|          347742|11.1333| NULL|       S|
|         10|       1|     2|Nasser, Mrs. Nich...|female|14.0|    1|    0|          237736|30.0708| NULL|       C|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [9]:
# 복합 조건: Pclass가 1이고 Age가 30 이상 (and 사용)
df_csv.where((F.col("Pclass") == 1) & (F.col("Age") >= 30)).limit(5).show()

+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-----+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|PC 17599|71.2833|  C85|       C|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|  113803|   53.1| C123|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|   17463|51.8625|  E46|       S|
|         12|       1|     1|Bonnell, Miss. El...|female|58.0|    0|    0|  113783|  26.55| C103|       S|
|         31|       0|     1|Uruchurtu, Don. M...|  male|40.0|    0|    0|PC 17601|27.7208| NULL|       C|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-----+--------+



Grouping & Aggregation: 그룹별 집계

In [10]:
# Pclass(객실 등급)별 승객 수, 평균 요금, 최대 나이 집계
df_csv.groupBy("Pclass").agg(
    F.count("*").alias("cnt"),                 # 각 등급별 승객 수
    F.avg("Fare").alias("avg_fare"),     # 각 등급별 평균 요금
    F.max("Age").alias("max_age"),       # 각 등급별 최대 나이
).orderBy("Pclass").show()               # Pclass 기준 오름차순 정렬 후 출력

+------+---+------------------+-------+
|Pclass|cnt|          avg_fare|max_age|
+------+---+------------------+-------+
|     1|216| 84.15468749999992|   80.0|
|     2|184| 20.66218315217391|   70.0|
|     3|491|13.675550101832997|   74.0|
+------+---+------------------+-------+



In [11]:
# Sex별 생존자 수
df_csv.filter(F.col("Survived") == 1).groupBy("Sex").count().orderBy(F.desc("count")).show()

+------+-----+
|   Sex|count|
+------+-----+
|female|  233|
|  male|  109|
+------+-----+



두 DataFrame의 Join

In [12]:
# Pclass 설명용 작은 테이블 생성
# 객실 등급 정보를 담은 DataFrame 생성 (등급 번호와 라벨 매핑)
class_info = spark.createDataFrame(
    [(1, "1st"), (2, "2nd"), (3, "3rd")],  # 데이터: (등급 번호, 등급 라벨) 튜플 리스트
    ["Pclass", "ClassLabel"]                # 컬럼명: Pclass, ClassLabel
)
class_info.toPandas()

,Pclass,ClassLabel
0,1,1st
1,2,2nd
2,3,3rd


Pclass 컬럼을 기준으로 두 DataFrame을 내부 조인 (양쪽에 모두 존재하는 데이터만 포함)

In [13]:
# df_csv와 class_info를 Pclass 기준으로 조인 (inner)
joined = df_csv.join(class_info, on="Pclass", how="inner")
joined.select("PassengerId", "Pclass", "ClassLabel", "Fare").limit(10).toPandas()

,PassengerId,Pclass,ClassLabel,Fare
0,890,1,1st,30.0000
1,888,1,1st,30.0000
2,880,1,1st,83.1583
3,873,1,1st,5.0000
4,872,1,1st,52.5542
5,868,1,1st,50.4958
6,863,1,1st,25.9292
7,858,1,1st,26.5500
8,857,1,1st,164.8667
9,854,1,1st,39.4000


CSV → Parquet 변환 저장

In [14]:
# DataFrame을 Parquet 형식으로 저장 (기존 파일이 있으면 덮어쓰기)
df_csv.write.mode("overwrite").parquet(OUTPUT_PARQUET)

Parquet 읽기 (전체)

In [15]:
# Parquet 파일을 DataFrame으로 읽기
df_pq = spark.read.parquet(OUTPUT_PARQUET)

# 상위 5개 행 출력
df_pq.limit(5).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

컬럼 프루닝: 필요한 컬럼만 선택 후 읽기 (Parquet은 컬럼 단위 저장이라 유리)

In [16]:
# Parquet 파일에서 특정 컬럼만 선택하여 읽기 (컬럼 프루닝)
df_pq_pruned = spark.read.parquet(OUTPUT_PARQUET).select("Survived", "Pclass", "Sex", "Age", "Fare")

# 상위 5개 행 출력
df_pq_pruned.limit(5).show()

+--------+------+------+----+-------+
|Survived|Pclass|   Sex| Age|   Fare|
+--------+------+------+----+-------+
|       0|     3|  male|22.0|   7.25|
|       1|     1|female|38.0|71.2833|
|       1|     3|female|26.0|  7.925|
|       1|     1|female|35.0|   53.1|
|       0|     3|  male|35.0|   8.05|
+--------+------+------+----+-------+

